In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
test_url = 'https://www.ravelry.com/patterns/library/love-note'
result = requests.get(test_url)
#if result.status_code == 200:
soup = BeautifulSoup(result.content, 'html.parser')

In [14]:
photo_gallery = soup.find('div', class_ = 'photo_gallery_container').find_all('img')

In [15]:
photo_gallery[0]

<img class="lazy_srcset lazy_srcset--loading" data-lazy-srcset="https://images4-g.ravelrycache.com/uploads/tincanknits/624440924/TCK-lovenote-01a_small2.jpg 1x, https://images4-f.ravelrycache.com/uploads/tincanknits/624440924/TCK-lovenote-01a_medium2.jpg 2x" height="150.0vw" id="hero_photo_84581553" onclick="R.photos.responsiveFullscreen(this); return false;" src="https://images4-f.ravelrycache.com/uploads/tincanknits/624440924/TCK-lovenote-01a_thumbnail.jpg"/>

In [17]:
photo_gallery[0]['src']

'https://images4-f.ravelrycache.com/uploads/tincanknits/624440924/TCK-lovenote-01a_thumbnail.jpg'

OK, code to get lists of photo gallery links for a given page is

In [18]:
[image['src'] for image in soup.find('div', class_ = 'photo_gallery_container').find_all('img')]

['https://images4-f.ravelrycache.com/uploads/tincanknits/624440924/TCK-lovenote-01a_thumbnail.jpg',
 'https://images4-g.ravelrycache.com/uploads/tincanknits/624440924/TCK-lovenote-01a_small2.jpg',
 'https://images4-f.ravelrycache.com/uploads/tincanknits/624440921/TCK-lovenote-12a_small2.jpg',
 'https://images4-f.ravelrycache.com/uploads/tincanknits/624440936/TCK-lovenote-18a_small2.jpg',
 'https://images4-g.ravelrycache.com/uploads/tincanknits/624440943/TCK-lovenote-16a_small2.jpg',
 'https://images4-g.ravelrycache.com/uploads/tincanknits/624440920/TCK-lovenote-14a_small2.jpg',
 'https://images4-g.ravelrycache.com/uploads/tincanknits/624440929/TCK-lovenote-10a_small2.jpg',
 'https://images4-g.ravelrycache.com/uploads/tincanknits/624440923/TCK-lovenote-07a_small2.jpg',
 'https://images4-f.ravelrycache.com/uploads/tincanknits/624440931/TCK-lovenote-06a_small2.jpg']

"Projects" page for a given pattern is same as pattern link but with ```/people``` appended to it.

Now want to get several photos from the projects page. Since there are also in progress photos, let's look at the finished projects page. ```/people?view=cards&search=&view=cards&statuses%5B%5D=finished```

Unexpected problem here: some people only have photos of the balls/skeins of yarn they plan on using to make the sweater, not the finished item. Possible solution: get the photo neural net to identify balls of yarn and drop those photos?

Getting thumbnails for first projects page:

In [23]:
people_url = 'https://www.ravelry.com/patterns/library/love-note/people?view=cards&search=&view=cards&statuses%5B%5D=finished'
result = requests.get(people_url)
people_soup = BeautifulSoup(result.content, 'html.parser')

In [30]:
soup.find('div', class_ = 'photo_frame photo_gallery__frame real_photo').find('img')

<img alt="" class="width_lock" data-image-height="213" data-image-width="320" data-photo-id="84581553" src="https://images4-g.ravelrycache.com/uploads/tincanknits/624440924/TCK-lovenote-01a_small2.jpg" style="left: -55px;"/>

In [31]:
soup.find_all('div', class_ = 'photo_frame photo_gallery__frame real_photo')

[<div class="photo_frame photo_gallery__frame real_photo"><div class="photo photo_gallery__photo full_height_photo zoomable_photo" id="photo_84581553"><picture><source media="(max-width: 479px)" srcset="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7"/><img alt="" class="width_lock" data-image-height="213" data-image-width="320" data-photo-id="84581553" src="https://images4-g.ravelrycache.com/uploads/tincanknits/624440924/TCK-lovenote-01a_small2.jpg" style="left: -55px;"/></picture></div></div>,
 <div class="photo_frame photo_gallery__frame real_photo"><div class="photo photo_gallery__photo zoomable_photo" id="photo_84581554"><picture><source media="(max-width: 479px)" srcset="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7"/><img alt="" class="height_lock" data-image-height="320" data-image-width="213" data-photo-id="84581554" src="https://images4-f.ravelrycache.com/uploads/tincanknits/624440921/TCK-lovenote-12a_small2.jpg"

In [32]:
# Community image thumbnails
[card.find('img')['src'] for card in soup.find_all('div', class_ = 'photo_frame photo_gallery__frame real_photo')]

['https://images4-g.ravelrycache.com/uploads/tincanknits/624440924/TCK-lovenote-01a_small2.jpg',
 'https://images4-f.ravelrycache.com/uploads/tincanknits/624440921/TCK-lovenote-12a_small2.jpg',
 'https://images4-f.ravelrycache.com/uploads/tincanknits/624440936/TCK-lovenote-18a_small2.jpg',
 'https://images4-g.ravelrycache.com/uploads/tincanknits/624440943/TCK-lovenote-16a_small2.jpg',
 'https://images4-g.ravelrycache.com/uploads/tincanknits/624440920/TCK-lovenote-14a_small2.jpg',
 'https://images4-g.ravelrycache.com/uploads/tincanknits/624440929/TCK-lovenote-10a_small2.jpg',
 'https://images4-g.ravelrycache.com/uploads/tincanknits/624440923/TCK-lovenote-07a_small2.jpg',
 'https://images4-f.ravelrycache.com/uploads/tincanknits/624440931/TCK-lovenote-06a_small2.jpg']

8 user-submitted pictures (probably at least 4 of them the actual garment) will be enough for the test run.

Now to take a step back and scrape pattern links from the search page.

In [35]:
search_url = 'https://www.ravelry.com/patterns/search#craft=crochet%7Cknitting&pc=sweater&sort=best&view=captioned_thumbs'
result = requests.get(search_url)
search_soup = BeautifulSoup(result.content, 'html.parser')

search_soup.find('div')

<div class="splash_page">
<div class="splash_page__motion_controls">
<a class="splash_page__motion_controls__play" data-action="play" href="#"><img alt="" class="icon_16 o-icon--play o-icon o-icon--xs" src="https://style-cdn.ravelrycache.com/images/assets/icons/player-controls/play.svg?v=15"/> Play</a>
<a class="splash_page__motion_controls__pause" data-action="pause" href="#"><img alt="" class="icon_16 o-icon--pause o-icon o-icon--xs" src="https://style-cdn.ravelrycache.com/images/assets/icons/player-controls/pause.svg?v=15"/> Pause</a>
</div>
<div class="splash_page__image">
<div class="splash_page__image__background">
<div class="splash_page__logo on_splash_page_transition">
<img src="https://style-cdn.ravelrycache.com/images/assets/logos/primary.svg"/>
</div>
<div class="splash_page__balloon on_splash_page_transition" id="splash_page_balloon_alpaca">
<img height="1228" src="https://style-cdn.ravelrycache.com/images/assets/splash/splash_balloon-alpaca-8bpp.png" width="682"/>
</div>


Search page requires login. Hmm. They have an API, which I'm going to request access to. In the meantime, fun with logins!

```<label for="user_login">Username</label>
<input autofocus="autofocus" id="user_login" name="user[login]" size="30" type="text" value="" style="background-image: url(&quot;data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR4nGP6zwAAAgcBApocMXEAAAAASUVORK5CYII=&quot;); cursor: auto;">```

```<label for="user_password">Password</label>
<input id="user_password" name="user[password]" size="30" type="password" style="background-image: url(&quot;data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR4nGP6zwAAAgcBApocMXEAAAAASUVORK5CYII=&quot;); cursor: auto;">```

```<meta content="BrjS4V87lBeD/pLeSy3xGn3mtGSMIQx2UKxE7bBeAV0=" id="authenticity-token" name="authenticity-token">```


In [54]:
from lxml import html

login = {'user_login': 'cheshious@gmail.com', 
        'user_password': 'angel1ne', 
        'authenticity_token': 'BrjS4V87lBeD/pLeSy3xGn3mtGSMIQx2UKxE7bBeAV0='}
session_requests = requests.session()
login_url = 'https://www.ravelry.com/account/login'
result = session_requests.get(login_url)

soup = BeautifulSoup(result.content, 'html.parser')
authenticity_token = soup.find('input')['value']
login['authenticity-token'] = authenticity_token

result = session_requests.post(login_url, 
                               data = login, 
                               headers = dict(referer=login_url))

In [63]:
login['user[login]'] = 'cheshious@gmail.com'
login['user[password]'] = 'angel1ne'

In [67]:
with session_requests as session:
    result = session.get(login_url)
    soup = BeautifulSoup(result.content, 'html.parser')
    authenticity_token = soup.find('input')['value']
    login['authenticity-token'] = authenticity_token
    result = session_requests.post(login_url, data = login)

In [68]:
result.status_code

200

In [70]:
soup = BeautifulSoup(result.content, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>Ravelry</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script>window.NREUM||(NREUM={});NREUM.info={"beacon":"bam-cell.nr-data.net","errorBeacon":"bam-cell.nr-data.net","licenseKey":"75f4b6d841","applicationID":"13630","transactionName":"Il8PEkVfCFpSRh9RAQJfFAhDHwhZUF1e","queueTime":0,"applicationTime":18,"agent":""}</script>
<script>(window.NREUM||(NREUM={})).loader_config={xpid:"VwcOU0VSCwUDVA==",licenseKey:"75f4b6d841",applicationID:"13630"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(t){try{c.console&&console.log(t)}catch(e){}}var i,o=t("ee"),a=t(23),c={};try{i=localStor

And they've got some anti-bot measures as well! Dang!

I'm going to cheat and right-click on some image links to get pattern pages I can scrape and play around with while I try to figure out their API.

In [71]:
pattern_links = ['https://www.ravelry.com/patterns/library/stripes-32', 
                'https://www.ravelry.com/patterns/library/love-note', 
                'https://www.ravelry.com/patterns/library/soldotna-crop', 
                'https://www.ravelry.com/patterns/library/the-weekender-5', 
                'https://www.ravelry.com/patterns/library/ranunculus-3', 
                'https://www.ravelry.com/patterns/library/flax', 
                'https://www.ravelry.com/patterns/library/zweig', 
                'https://www.ravelry.com/patterns/library/pull-camille', 
                'https://www.ravelry.com/patterns/library/navelli', 
                'https://www.ravelry.com/patterns/library/soundtrack-2', 
                'https://www.ravelry.com/patterns/library/felix-pullover', 
                'https://www.ravelry.com/patterns/library/forestland', 
                'https://www.ravelry.com/patterns/library/super-simple-summer-sweater', 
                'https://www.ravelry.com/patterns/library/no-frills-sweater', 
                'https://www.ravelry.com/patterns/library/cumulus-blouse', 
                'https://www.ravelry.com/patterns/library/ursa-3', 
                'https://www.ravelry.com/patterns/library/fonda', 
                'https://www.ravelry.com/patterns/library/humulus-2', 
                'https://www.ravelry.com/patterns/library/mjuk-hettegenser-herre', 
                'https://www.ravelry.com/patterns/library/misurina', 
                'https://www.ravelry.com/patterns/library/city-limits-2', 
                'https://www.ravelry.com/patterns/library/yume', 
                'https://www.ravelry.com/patterns/library/so-faded', 
                'https://www.ravelry.com/patterns/library/the-throwover', 
                'https://www.ravelry.com/patterns/library/rift-7', 
                'https://www.ravelry.com/patterns/library/felix-cardigan', 
                'https://www.ravelry.com/patterns/library/shifty-2', 
                'https://www.ravelry.com/patterns/library/forager-4', 
                'https://www.ravelry.com/patterns/library/elton-3', 
                'https://www.ravelry.com/patterns/library/sunday-sweater-7', 
                'https://www.ravelry.com/patterns/library/the-throwback', 
                'https://www.ravelry.com/patterns/library/wool--honey', 
                'https://www.ravelry.com/patterns/library/comfort-fade-cardi', 
                'https://www.ravelry.com/patterns/library/magnolia-chunky-cardigan', 
                'https://www.ravelry.com/patterns/library/nurtured', 
                'https://www.ravelry.com/patterns/library/sipila', 
                'https://www.ravelry.com/patterns/library/sorrel-15', 
                'https://www.ravelry.com/patterns/library/trescao-jumper', 
                'https://www.ravelry.com/patterns/library/gardengate', 
                'https://www.ravelry.com/patterns/library/spark-5', 
                'https://www.ravelry.com/patterns/library/cozy-classic-raglan', 
                'https://www.ravelry.com/patterns/library/tecumseh-2', 
                'https://www.ravelry.com/patterns/library/the-weekender-light', 
                'https://www.ravelry.com/patterns/library/turtle-dove-3', 
                'https://www.ravelry.com/patterns/library/fog-whisperer-cardigan', 
                'https://www.ravelry.com/patterns/library/pink-velvet']

# First page of results at https://www.ravelry.com/patterns/search#craft=crochet%7Cknitting&availability=free%7Conline%7Cravelry&pc=sweater&photo=yes&sort=best&view=captioned_thumbs
# Knit or crochet, available online, sweater, has photos

In [72]:
from fake_useragent import UserAgent

ua = UserAgent()
user_agent = {'User-agent': ua.random}
search_stem = 'https://www.ravelry.com/patterns/search#craft=crochet%7Cknitting&availability=free%7Conline%7Cravelry&photo=yes&pc=sweater&sort=best&view=captioned_thumbs&page='
session_requests = requests.Session()
with session_requests as session:
    result = session.get(login_url, headers = user_agent)
    soup = BeautifulSoup(result.content, 'html.parser')
    authenticity_token = soup.find('input')['value']
    login['authenticity-token'] = authenticity_token
    session_requests.post(login_url, data = login)
    result = session.get(search_stem + '2', headers = user_agent)

In [73]:
soup = BeautifulSoup(result.content, 'html.parser')

In [74]:
soup

<!DOCTYPE html>

<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>Ravelry</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script>window.NREUM||(NREUM={});NREUM.info={"beacon":"bam-cell.nr-data.net","errorBeacon":"bam-cell.nr-data.net","licenseKey":"75f4b6d841","applicationID":"13630","transactionName":"Il8PEkVfCFpSRh9RAQJfFAhDHwhZUF1e","queueTime":2,"applicationTime":18,"agent":""}</script>
<script>(window.NREUM||(NREUM={})).loader_config={xpid:"VwcOU0VSCwUDVA==",licenseKey:"75f4b6d841",applicationID:"13630"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(t){try{c.console&&console.log(t)}catch(e){}}var i,o=t("ee"),a=t(23),c={};try{i=localStor

In [75]:
# Getting pattern name from url:
import regex as re
import time

pattern_scrapes = {}
for link in pattern_links:
    user_agent = {'User-agent': ua.random}
    name = re.search('(?<=library/)[-A-z0-9]+', link)
    result = requests.get(link, headers = user_agent)
    pattern_scrapes[name] = result
    time.sleep(30)

In [76]:
print(len(pattern_scrapes), len(pattern_links))

46 46


In [93]:
soup_dict = {}
for key in pattern_scrapes.keys():
    # key is currently a regex.Match object; getting string of what matched
    soup_dict[key.group()] = BeautifulSoup(pattern_scrapes[key].content, 'html.parser')

In [94]:
pattern_dict = {}
for key in soup_dict.keys():
    pattern_dict[key] = {}
    pattern_dict[key]['gallery'] = [image['src'] for image in soup_dict[key].find('div', class_ = 'photo_gallery_container').find_all('img')]

In [96]:
people_scrapes = {}
# Getting community photos of same project; first page only
for link in pattern_links:
    people_link = link + '/people?view=cards&search=&view=cards&statuses%5B%5D=finished'
    user_agent = {'User-agent': ua.random}
    name = re.search('(?<=library/)[-A-z0-9]+', link).group()
    result = requests.get(people_link, headers = user_agent)
    people_scrapes[name] = result
    time.sleep(30)

In [97]:

# Getting second page of projects to test neural net
people_scrapes2 = {}
# Getting community photos of same project; first page only
for link in pattern_links:
    people_link = link + '/people?page=2&search=&statuses%5B%5D=finished&view=cards'
    user_agent = {'User-agent': ua.random}
    name = re.search('(?<=library/)[-A-z0-9]+', link).group()
    result = requests.get(people_link, headers = user_agent)
    people_scrapes2[name] = result
    time.sleep(30)

In [101]:
# Turning raw results of community pages to BeautifulSoup objects
people_soup = {}
for key in people_scrapes.keys():
    people_soup[key] = BeautifulSoup(people_scrapes[key].content, 'html.parser')
people_soup2 = {}
for key in people_scrapes2.keys():
    people_soup2[key] = BeautifulSoup(people_scrapes2[key].content, 'html.parser')

In [102]:
# Getting links from community pages; appending first to links list in dict and putting second in its own list
for key in pattern_dict.keys():
    pattern_dict[key]['gallery'] += [card.find('img')['src'] for card in people_soup[key].find_all('div', class_ = 'photo_frame photo_gallery__frame real_photo')]
    pattern_dict[key]['test links'] = [card.find('img')['src'] for card in people_soup2[key].find_all('div', class_ = 'photo_frame photo_gallery__frame real_photo')]
    

In [128]:
import os.path
import urllib.request 

test_pic = urllib.request.urlretrieve('https://images4-f.ravelrycache.com/uploads/dreareneeknits/743362891/Custom-8_thumbnail.jpg', 'test.jpg')


In [134]:
with open('text.txt', 'w') as fp:
    fp.write('I am a test')

In [129]:
os.chdir('proj05')

In [133]:
os.getcwd()

'/Users/chesh1/Documents/DS/onl20_ds4/curriculum/project-05/proj05'

In [ ]:
for pattern in pattern_dict.keys():
    train_count = 1
    test_count = 100
    for link in pattern_dict[pattern]['gallery']:
        file_name = pattern + '_' + str(train_count).zfill(3) + '.jpg'
        file_path = os.path.join('img/', file_name)
        pic = urllib.request.urlretrieve(link, file_path)
        train_count += 1
        time.sleep(30)
    for link in pattern_dict[pattern]['test']:
        file_name = pattern + '_' + str(test_count).zfill(3) + '.jpg'
        file_path = os.path.join('img/test/')
        pic = urllib.request.urlretrieve(link, file_name)
        test_count += 1